In [25]:
!pip install opencv-python-headless numpy --user
!pip install tensorflow --user
!pip install keras --user

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/ad/6e/1bfe367855dd87467564f7bf9fa14f3b17889988e79598bc37bf18f5ffb6/tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata (4.8 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using 

In [26]:
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from sklearn.model_selection import train_test_split

# Load a pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model that includes the VGG16 base model followed by a Flatten layer
model = Model(inputs=base_model.input, outputs=Flatten()(base_model.layers[-1].output))

# Load your video file
video_capture = cv2.VideoCapture("input2.mp4")

# Initialize lists to store features and labels
features = []
labels = []

while True:
    # Capture a frame from the video
    ret, frame = video_capture.read()
    if not ret:
        break

    # Preprocess the frame for the VGG16 model
    frame = cv2.resize(frame, (224, 224))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = preprocess_input(frame)

    # Extract features from the frame using the VGG16 model
    features.append(model.predict(np.expand_dims(frame, axis=0))[0])

    # Assuming you have some way to determine whether the frame contains a car or not
    # In this example, I'll assume that if the frame number is even, it contains a car.
    label = 1 if video_capture.get(cv2.CAP_PROP_POS_FRAMES) % 2 == 0 else 0
    labels.append(label)

# Convert lists to NumPy arrays
features = np.array(features)
labels = np.array(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Now you can use x_train and y_train to train a perceptron or any other classifier
# to classify whether an image contains a car or not.
# Replace this part with your specific classification code.


1/1 [==============================] - 0s 262ms/step


KeyboardInterrupt: 

In [30]:
import cv2
import numpy as np
import os

# Create directories for storing images
os.makedirs("Car", exist_ok=True)
os.makedirs("NotCar", exist_ok=True)

# Hyperparameters
learning_rate = 0.1  # Smaller learning rate
epochs = 100

# Initialize weights and bias with smaller values
input_dim = 3  # RGB channels
weights = np.random.rand(input_dim) * 0.001
bias = np.random.rand() * 0.001

# Function to predict using the Perceptron
def predict(x):
    return 1 if np.dot(x, weights) + bias > 0 else 0

# Function to train the Perceptron
def train(x_train, y_train):
    global weights, bias
    for _ in range(epochs):
        for i in range(len(x_train)):
            y_pred = predict(x_train[i])
            error = y_train[i] - y_pred
            weights += learning_rate * error * x_train[i]
            bias += learning_rate * error

# Load the video
cap = cv2.VideoCapture('input2.mp4')

# Initialize lists to store features and labels
features = []
labels = []

# Loop through frames
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to a fixed size (adjust as needed)
    frame = cv2.resize(frame, (100, 100))
    
    # Extract RGB values
    r, g, b = np.mean(frame[:, :, 0]), np.mean(frame[:, :, 1]), np.mean(frame[:, :, 2])
    
    # Normalize values to [0, 1]
    r, g, b = r / 255.0, g / 255.0, b / 255.0
    
    # Create input vector
    input_vector = np.array([r, g, b])
    
    # Predict using the Perceptron
    result = predict(input_vector)
    
    if result == 1:
        cv2.imwrite(f'Car/frame{frame_count}.jpg', frame)
    else:
        cv2.imwrite(f'NotCar/frame{frame_count}.jpg', frame)
    
    features.append(input_vector)
    labels.append(result)
    
    frame_count += 1

cap.release()
cv2.destroyAllWindows()

# Convert lists to NumPy arrays
features = np.array(features)
labels = np.array(labels)

# Train the Perceptron
train(features, labels)

# Save weights and bias to answer.txt
with open('answer.txt', 'w') as f:
    f.write(f'Weights: {weights.tolist()}\n')
    f.write(f'Bias: {bias}\n')
